In [26]:
import pandas as pd
import gzip
import mysql.connector
import nltk
from langdetect import detect  # 用于检测文本语言

# nltk.download()

In [2]:
def read_data_from_mysql(table_name):
    # 设置本地MySQL数据库连接信息
    host = 'localhost'  # 替换为本地MySQL数据库的主机名
    user = 'root'  # 替换为数据库用户名
    password = '991208louise'  # 替换为数据库密码
    database = 'amazon'  # 替换为数据库名称

    # 建立数据库连接
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    print(f'Connected to MySQL')
    # Read data from MySQL into a DataFrame
    query = f"SELECT * FROM {table_name};"
    df = pd.read_sql(query, conn)
    print(f'load {table_name} data from MySQL!')

    # Close the database connection
    conn.close()

    return df

# *Function*

# **Load MySQL**

In [3]:
def saved_mysql(df, table_name):
    # 设置本地MySQL数据库连接信息
    host = 'localhost'  # 替换为本地MySQL数据库的主机名
    user = 'root'  # 替换为数据库用户名
    password = '991208louise'  # 替换为数据库密码
    database = 'amazon'  # 替换为数据库名称

    # 建立数据库连接
    conn = mysql.connector.connect(host=host, user=user, password=password, database=database)
    # 检查表是否存在
    table_exists = False
    with conn.cursor() as cursor:
        check_table_sql = f"SHOW TABLES LIKE '{table_name}';"
        cursor.execute(check_table_sql)
        table_exists = cursor.fetchone() is not None
        # print(f"Table '{table_name}' exists: {table_exists}")
    # if table_exists:
    #     print(f"Table '{table_name}' already exists. Drop it...")
    #     with conn.cursor() as cursor:
    #         # Move the del_table query execution inside the 'with' block
    #         del_table = f"-- DROP TABLE {table_name};"
    #         cursor.execute(del_table)
    #         # table_exists 设置为反面
    #         table_exists=False

    if not table_exists:
        print(f"Table '{table_name}' does not exist. Creating new table...")

            # 定义reviewText列的数据类型为LONGTEXT，helpful列的数据类型为INT
        column_types = {
                'reviewerID': 'VARCHAR(255)',
                'asin': 'VARCHAR(255)',
                'overall': 'FLOAT',
                'reviewTime': 'DATE',
                'reviewText': 'LONGTEXT',
                'helpful_vote': 'INT',
                'total_vote': 'INT',
                'helpfulness_score': 'FLOAT'
        }

        # 构建创建表的SQL语句
        create_table_sql = f"CREATE TABLE {table_name} ("
        for col_name, col_type in column_types.items():
            create_table_sql += f"{col_name} {col_type}, "
        create_table_sql = create_table_sql.rstrip(', ')  # 去除最后一个逗号和空格
        create_table_sql += ");"

        # 创建表
        with conn.cursor() as cursor:
            cursor.execute(create_table_sql)
        print(f"Table '{table_name}' created.")

    # 将DataFrame数据插入数据库表
    with conn.cursor() as cursor:
        for _, row in df.iterrows():
            insert_sql = f"INSERT INTO {table_name} ({', '.join(df.columns)}) VALUES ({', '.join(['%s'] * len(df.columns))})"
            cursor.execute(insert_sql, tuple(row))

    # 提交数据
    conn.commit()

    # 关闭数据库连接
    conn.close()
    # print(f'Finished saving {table_name} to MySQL!')


# **Load Dataset**

In [4]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

In [5]:
# for long dataset(>1000000)
def getDF_reviews(path, columns_keep,file_name):
    batch_size = 1000
    i = 0
    # Create an empty list to store individual DataFrames
    dataframes =[]
    dataframes_list=[]
    df = {}
    saved=0
    for d in parse(path):
        df[i] = d
        i += 1
        if i ==13121199:
            file_name=f'{file_name}_1'
            print(f'Processed {i} records')

        if i % batch_size == 0:
            df = pd.DataFrame.from_dict(df, orient='index')
            df = df[columns_keep]
            # print(f'The dataframe include: {df.columns}')
            df=data_format(df)
            df=review_data_clean(df)
            saved_mysql(df, file_name)
            saved+=len(df)
            dataframes.append(df)
            df = {}
            if i % 1000000== 0:
                # print(f'Processed {i} records')
                dataframes_list.append(pd.concat(dataframes,ignore_index=True))
                print(f'\n================================================================\n  Saved total {len(dataframes_list)} dataframes to MySQL {file_name} : \n{saved}\n================================================================')
                # print(dataframes[1000:1005])
                dataframes=[]

    # Save any remaining records
    if len(dataframes) > 0:
        dataframes_list.append(pd.concat(dataframes,ignore_index=True))
        print(f'\n dataframes_list append {len(dataframes)} dataframe chuck as one dataframes ====================================================')
    if len(df)>0:
        print(f'Processed {i} records')
        df = pd.DataFrame.from_dict(df, orient='index')
        df = df[columns_keep]
        df=data_format(df)
        df=review_data_clean(df)
        saved_mysql(df, file_name)
        print(f'Saved total {(saved+len(df))} records to MySQL')
        dataframes_list.append(df)
    # print(f'Finished saving {file_name} to MySQL!')
    print(f'Total {len(dataframes_list)} dataframes generated')
    return dataframes_list


# change helpfulness format
def data_format(df):
    # change helpfulness format
    df['helpful_vote'] = df['helpful'].apply(lambda x: x[0])
    df['total_vote'] = df['helpful'].apply(lambda x: x[1])
    # 计算helpful score并在total Vote为0时设置为0
    df['helpfulness_score'] = df.apply(lambda row: row['helpful_vote'] / row['total_vote'] if row['total_vote'] != 0 else 0, axis=1)

    # 使用drop方法删除原始helpful列
    df.drop(columns=['helpful'], inplace=True)
    # Convert 'reviewTime' column to MySQL-compatible DATE format
    df['reviewTime'] = pd.to_datetime(df['reviewTime'], format='%m %d, %Y').dt.strftime('%Y-%m-%d')

    return df

***identify columns***

In [6]:
review_columns=['reviewerID','asin','overall','reviewTime','reviewText','helpful']
content_columns_reviews = ['reviewText']

# **Data Clean**

***delete NaN***

In [7]:
import re
# 删除reviews存在NaN的行
def review_data_clean(df):
    # 删除reviews存在NaN的行
    df.dropna(subset=['reviewerID','asin','overall','reviewTime','reviewText','total_vote','helpfulness_score'], inplace=True)
    df = df[df['reviewText'].apply(lambda x: len(x) >0)]
    # 筛选出不包含非法字符的行
    df_cleaned = df[~df['reviewText'].apply(lan_check)]
    if len(df_cleaned) < len(df):
        # 合并df和df_cleaned，并标记来源
        merged_df = df.merge(df_cleaned, how='outer', indicator=True)
        # 筛选出df中df_cleaned没有的那行
        not_in_df_cleaned = merged_df[merged_df['_merge'] == 'left_only']
        # 设置显示列的最大宽度为None（表示不截断文本）
        # pd.options.display.max_colwidth = None
        print(not_in_df_cleaned['reviewText'])
    return df_cleaned


def lan_check(text):
    try:
        if detect(text) == 'en':
            return True
        else:
            print(f'Not English!!!!!!!!!!!!!:  {text}')
            return False
    except:
        print('\n======\nHave Question:\n  text')
        return True



# **Statistic**

***统计reviews的最大长度，最小长度，平均长度,中位数以及3/4位数***

In [8]:

# 计算 reviewText, summary的最大长度，最小长度，平均长度
# import matplotlib.pyplot as plt
import statistics

# 假设books_revibews是你的数据框
def review_len(df):
    num_rows, num_cols = df.shape
    # 首先，计算每个reviewText的长度，并将结果存储在新的列'reviewText_length'中
    # Calculate the length of each 'reviewText' and store in a list
    review_lengths = df['reviewText'].apply(lambda x: len(x)).tolist()

    # Sort the 'review_lengths' list in ascending order
    sorted_review_lengths = sorted(review_lengths)
    # Find the maximum length
    max_length = max(sorted_review_lengths)

    # Find the minimum length
    min_length = min(sorted_review_lengths)

    # Find the average length (mean)
    average_length = sum(sorted_review_lengths) / len(sorted_review_lengths)

    # Find the median length
    median_length = statistics.median(sorted_review_lengths)
    # Find the 3/4 quartile (75th percentile)
    quantile_3_4 = statistics.quantiles(sorted_review_lengths, n=4)[-1]

    all_sentences = df['reviewText'].apply(nltk.sent_tokenize)
    # 计算句子的总个数
    total_sentences = all_sentences.apply(len).sum()
    # 计算平均句子个数
    avg_sentences = all_sentences.apply(len).mean()

    print(f'Num of rows: {num_rows}')
    print('====      ')
    print(f'Avg sentences: {avg_sentences}')
    print(f'Total sentences: {total_sentences}')
    print('====      ')
    print(f'Max text length: {max_length}')
    print(f'Min text length: {min_length}')
    print(f'Avg text length: {average_length}')
    print(f'Mid text length: {median_length}')
    print(f'q75 text length: {quantile_3_4}')
    print("=====================================")


***data investigate function***

In [9]:
def data_quality(df,content_columns):
    # 统计总行数和列数
    num_rows, num_cols = df.shape
    # 列出所有列名
    column_names = df.columns.tolist()
    # 统计每列的缺失值数量
    missing_values = df.isnull().sum()

    count_all_nan = df[df[content_columns].isnull().all(axis=1)].shape[0]
    print(' data of specified columns is NaN:',count_all_nan)

    # 创建数据质量报告的数据框架
    report = pd.DataFrame({
        '列名': column_names,
        'num_rows': num_rows,
        '缺失值数量': missing_values,
        '数据类型': df.dtypes,

    })
    print(report)

# *2. Review dataset*
** not in filtered_product**

# **Data Filter**
***Filter with asin***

In [10]:
# def filter_by_asin(df, file_name):
#     # Step 1: Read the ASINs to be removed from the file into a list
#     with open(file_name, 'r') as file:
#         lines = file.readlines()
#         asins_to_remove = [line.strip() for line in lines if line.strip()]
#
#     # Step 2: Create a boolean mask based on 'asin' column to identify rows to be removed
#     asin_mask = ~df['asin'].isin(asins_to_remove)
#
#     # Step 3: Create a new DataFrame with rows not matching the ASINs in the list
#     df_filtered = df[asin_mask]
#
#     return df_filtered

***Filter with timeline***

In [11]:
def filter_by_timeline(df):
    # 筛选出2004年7月之后的数据
    cutoff_date = pd.to_datetime('2009-07-01')
    # Convert the 'reviewTime' column to Timestamp
    df['reviewTime'] = pd.to_datetime(df['reviewTime'])

    df = df[df['reviewTime'] >= cutoff_date]
    return df

**Base on Vote**

***统计vote为0的数量,vote≥2***

In [12]:
# 统计 vote 的数量和比例
def vote_analysis(df):
    num_rows, num_cols = df.shape
    vote_zeros = df[df['total_vote']==0].shape[0]
    vote_zeros_ratio = vote_zeros / num_rows

    # 统计 total vote > 2 的数量
    total_vote=df['total_vote']
    greater_than_2_count = (total_vote >= 5).sum()
    # 统计 helfpful vote 不为的数量、比例
    helpful_not_zeros=df[df['helpful_vote']>0].shape[0]
    helpful_not_zeros_1=df[df['helpful_vote']!=0].shape[0]
    if helpful_not_zeros != helpful_not_zeros_1:
        print('!!!!!!!!!!!data has problem!!!!!!!!!')
    print('=====================')
    print('total vote is 0: ', vote_zeros)
    print('     the ratio: ',vote_zeros_ratio)
    print('total vote>=2: ',greater_than_2_count)
    print('helpful vote is not 0: ',helpful_not_zeros)
    print('=====================\n')

    # return helpful_zeros,helpful_zeros_ratio,helpful_not_zeros,b_greater_than_3_count,b_greater_than_5_count

***vote>2***

In [13]:
def filter_by_vote(df):
    new_df = df[df['total_vote'] >= 5]
    return new_df

# ***Reviews: Cell phone and Accessories***

In [38]:
file_name='cell_phones_reviews_original'
cell_phones_reviews_list= getDF_reviews('data/reviews_Cell_Phones_and_Accessories.json.gz',review_columns,file_name)

KeyboardInterrupt: 

In [15]:
cell_phones_reviews_list[0].head()

,reviewerID,asin,overall,reviewTime,reviewText,helpful_vote,total_vote,helpfulness_score
0,A1EWN6KZ4HMLT7,011040047X,1.0,2012-08-14,"The case pictured is a soft violet color, but the case cover I received was a dark purple. While I'm sure the quality of the product is fine, the color is very different.",2,2,1.0
1,A1YX2RBMS1L9L,0110400550,5.0,2012-11-22,Saw this same case at a theme park store for 25 dollars. This is very good quality for a great price.,0,0,0.0
2,A180NNPPKWCCU0,0110400550,5.0,2013-07-18,case fits perfectly and I always gets compliments on it its hasn't cracked when I dropped it. wonderful and protective.,3,3,1.0
3,A3HVRXV0LVJN7,0110400550,5.0,2013-01-13,Best phone case ever . Everywhere I go I get a ton of compliments on it. It was in perfect condition as well.,4,4,1.0
4,A292527VPX98P8,0110400550,1.0,2012-11-26,It may look cute. This case started off pretty good the first couple of weeks. Then it started to slide off. It slid off one day I was in a parking lot phone fell face down and my glass shattered. TERRIBLE CASE!!!!!,0,1,0.0


In [16]:
for cell_phones_reviews in cell_phones_reviews_list:
    print('\n***The cell_phones_reviews after cleaned:***')
    # review_len(cell_phones_reviews)
    data_quality(cell_phones_reviews,content_columns_reviews)


***The cell_phones_reviews after cleaned:***
 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量     数据类型
reviewerID                reviewerID    999680      0   object
asin                            asin    999680      0   object
overall                      overall    999680      0  float64
reviewTime                reviewTime    999680      0   object
reviewText                reviewText    999680      0   object
helpful_vote            helpful_vote    999680      0    int64
total_vote                total_vote    999680      0    int64
helpfulness_score  helpfulness_score    999680      0  float64

***The cell_phones_reviews after cleaned:***
 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量     数据类型
reviewerID                reviewerID    999777      0   object
asin                            asin    999777      0   object
overall                      overall    999777      0  float64
reviewTime     

**Data filter**

**Filter with asin,timeline and vote**

In [17]:
cell_phones_reviews_filter_list=[]
for cell_phones_reviews in cell_phones_reviews_list:
    cell_phones_reviews_filter=filter_by_timeline(cell_phones_reviews)
    print('filter with timeline finished!')
    vote_analysis(cell_phones_reviews_filter)
    cell_phones_reviews_filter=filter_by_vote(cell_phones_reviews_filter)
    cell_phones_reviews_filter_list.append(cell_phones_reviews_filter)
    print('filter with vote finished!')

filter with timeline finished!
total vote is 0:  667246
     the ratio:  0.7295248028986553
total vote>=2:  124505
helpful vote is not 0:  206637

filter with vote finished!
filter with timeline finished!
total vote is 0:  767462
     the ratio:  0.7676462350964232
total vote>=2:  108174
helpful vote is not 0:  186527

filter with vote finished!
filter with timeline finished!
total vote is 0:  738683
     the ratio:  0.7388632826409643
total vote>=2:  123959
helpful vote is not 0:  204822

filter with vote finished!
filter with timeline finished!
total vote is 0:  183
     the ratio:  0.7379032258064516
total vote>=2:  26
helpful vote is not 0:  22

filter with vote finished!


In [18]:
for cell_phones_reviews in cell_phones_reviews_filter_list:
    print('\n******************\n')
    review_len(cell_phones_reviews)
    data_quality(cell_phones_reviews,content_columns_reviews)


******************

Num of rows: 124505
====      
Avg sentences: 6.479394401831252
Total sentences: 806717
====      
Max text length: 32384
Min text length: 5
Avg text length: 644.3291434078952
Mid text length: 379
q75 text length: 740.0
 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量            数据类型
reviewerID                reviewerID    124505      0          object
asin                            asin    124505      0          object
overall                      overall    124505      0         float64
reviewTime                reviewTime    124505      0  datetime64[ns]
reviewText                reviewText    124505      0          object
helpful_vote            helpful_vote    124505      0           int64
total_vote                total_vote    124505      0           int64
helpfulness_score  helpfulness_score    124505      0         float64

******************

Num of rows: 108174
====      
Avg sentences: 5.819087766006619
Total s

***Concatenate***

In [19]:
cell_phones_reviews_concatenated = pd.concat(cell_phones_reviews_filter_list, ignore_index=True)
cell_phones_reviews_concatenated['reviewTime'] = cell_phones_reviews_concatenated['reviewTime'].dt.strftime('%Y-%m-%d')
table_name='cell_phones_reviews_filter'
saved_mysql(cell_phones_reviews_concatenated,table_name)

Table 'cell_phones_reviews_filter' does not exist. Creating new table...
Table 'cell_phones_reviews_filter' created.


In [20]:
review_len(cell_phones_reviews_concatenated)
data_quality(cell_phones_reviews_concatenated,content_columns_reviews)

Num of rows: 356664
====      
Avg sentences: 5.924578875356078
Total sentences: 2113084
====      
Max text length: 32384
Min text length: 2
Avg text length: 568.4193975282058
Mid text length: 320.0
q75 text length: 643.0
 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量     数据类型
reviewerID                reviewerID    356664      0   object
asin                            asin    356664      0   object
overall                      overall    356664      0  float64
reviewTime                reviewTime    356664      0   object
reviewText                reviewText    356664      0   object
helpful_vote            helpful_vote    356664      0    int64
total_vote                total_vote    356664      0    int64
helpfulness_score  helpfulness_score    356664      0  float64


# ***Reviews: Electronics***

In [21]:
file_name='electronics_reviews_original'
electronics_reviews_list=getDF_reviews('data/reviews_Electronics.json.gz',review_columns,file_name)

Table 'electronics_reviews_original' does not exist. Creating new table...
Table 'electronics_reviews_original' created.
904    Having set up similar products before, performance, features, and company reputation were priorities for me more than easy setup.  This product has performed very well for me.  The built in 4-port switch handles a home office network very nicely, and this router has the other features that I was looking for.The easy setup though was impressive from my point of view.  This is the only product I have personally seen that does not require you to enter any numbers to get a PC to talk to it.  You do have to select the appropriate network settings and reboot your computer, but that is all.  The router provides you with a simple and clear Web-based interface to complete the setup.  The documentation is also clear and to the point.  Overall I can highly recommend this product.Asante has a very good reputation for technical support.  For a product like this that is so

In [22]:
for electronics_reviews in electronics_reviews_list:
    print('\n***The electronics_reviews after cleaned:***')
    # review_len(electronics_reviews)
    data_quality(electronics_reviews,content_columns_reviews)


***The electronics_reviews after cleaned:***
 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量     数据类型
reviewerID                reviewerID    999678      0   object
asin                            asin    999678      0   object
overall                      overall    999678      0  float64
reviewTime                reviewTime    999678      0   object
reviewText                reviewText    999678      0   object
helpful_vote            helpful_vote    999678      0    int64
total_vote                total_vote    999678      0    int64
helpfulness_score  helpfulness_score    999678      0  float64

***The electronics_reviews after cleaned:***
 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量     数据类型
reviewerID                reviewerID    999496      0   object
asin                            asin    999496      0   object
overall                      overall    999496      0  float64
reviewTime     

**Data filter**

In [23]:
electronics_reviews_filter_list=[]
for electronics_reviews in electronics_reviews_list:
    electronics_reviews_filter=filter_by_timeline(electronics_reviews)
    print('filter with timeline finished!')
    vote_analysis(electronics_reviews_filter)
    electronics_reviews_filter=filter_by_vote(electronics_reviews_filter)
    print('filter with vote finished!')
    electronics_reviews_filter_list.append(electronics_reviews_filter)

filter with timeline finished!
total vote is 0:  329979
     the ratio:  0.6595372152566662
total vote>=2:  85664
helpful vote is not 0:  140396

filter with vote finished!
filter with timeline finished!
total vote is 0:  452805
     the ratio:  0.649804185507862
total vote>=2:  128200
helpful vote is not 0:  203168

filter with vote finished!
filter with timeline finished!
total vote is 0:  580341
     the ratio:  0.6018236987506015
total vote>=2:  224563
helpful vote is not 0:  324791

filter with vote finished!
filter with timeline finished!
total vote is 0:  612472
     the ratio:  0.613304253872505
total vote>=2:  226401
helpful vote is not 0:  327088

filter with vote finished!
filter with timeline finished!
total vote is 0:  628799
     the ratio:  0.6292803995056218
total vote>=2:  211850
helpful vote is not 0:  309811

filter with vote finished!
filter with timeline finished!
total vote is 0:  641267
     the ratio:  0.6415800910844492
total vote>=2:  201903
helpful vote is no

In [24]:
for electronics_reviews in electronics_reviews_filter_list:
    print('\n******************\n')
    # review_len(electronics_reviews)
    data_quality(electronics_reviews,content_columns_reviews)


******************

 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量            数据类型
reviewerID                reviewerID     85664      0          object
asin                            asin     85664      0          object
overall                      overall     85664      0         float64
reviewTime                reviewTime     85664      0  datetime64[ns]
reviewText                reviewText     85664      0          object
helpful_vote            helpful_vote     85664      0           int64
total_vote                total_vote     85664      0           int64
helpfulness_score  helpfulness_score     85664      0         float64

******************

 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量            数据类型
reviewerID                reviewerID    128200      0          object
asin                            asin    128200      0          object
overall                      overall    1282

***Concatenate***

In [25]:
electronics_reviews_concatenated = pd.concat(electronics_reviews_filter_list, ignore_index=True)
electronics_reviews_concatenated['reviewTime'] = electronics_reviews_concatenated['reviewTime'].dt.strftime('%Y-%m-%d')
table_name='electronics_reviews_filter'
saved_mysql(electronics_reviews_concatenated,table_name)

Table 'electronics_reviews_filter' does not exist. Creating new table...
Table 'electronics_reviews_filter' created.


In [26]:
review_len(electronics_reviews_concatenated)

Num of rows: 1291534
====      
Avg sentences: 7.374018802447322
Total sentences: 9523796
====      
Max text length: 32703
Min text length: 2
Avg text length: 764.022096204978
Mid text length: 447.0
q75 text length: 905.0


In [27]:
data_quality(electronics_reviews_concatenated,content_columns_reviews)

 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量     数据类型
reviewerID                reviewerID   1291534      0   object
asin                            asin   1291534      0   object
overall                      overall   1291534      0  float64
reviewTime                reviewTime   1291534      0   object
reviewText                reviewText   1291534      0   object
helpful_vote            helpful_vote   1291534      0    int64
total_vote                total_vote   1291534      0    int64
helpfulness_score  helpfulness_score   1291534      0  float64


# ***Video Games***

In [28]:
file_name='video_games_reviews_original'
video_games_reviews_list=getDF_reviews('data/reviews_Video_Games.json.gz',review_columns,file_name)

Table 'video_games_reviews_original' does not exist. Creating new table...
Table 'video_games_reviews_original' created.
611    I bought this game quite a while ago, but I hadn't spent much time playing with it--I tried it once, but I wasn't able to easily figure out how the game worked and the manual is lacking in several areas.  Now that I've spent a couple of days poking at it...There are two really major problems with the game: you can't save incomplete games, and a really obnoxious lockup problem, at least on my PS2.The inability to save in-progress games is understandable--it would be a challenge to get even a single map to fit on a memory card.  128K isn't much room for this sort of game.The lockup problem happens randomly when I'm in one of the menus. It's happened three times in a row after I've spent an hour or two building a really complex rail network (in one of the Australian scenarios), and thanks to the lack of a save feature that means all the work I've done is lost.  A

In [29]:
for video_games_reviews in video_games_reviews_list:
    print('\n***The video_games_reviews after cleaned:***')
    # review_len(video_games_reviews)
    data_quality(video_games_reviews,content_columns_reviews)


***The video_games_reviews after cleaned:***
 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量     数据类型
reviewerID                reviewerID    999754      0   object
asin                            asin    999754      0   object
overall                      overall    999754      0  float64
reviewTime                reviewTime    999754      0   object
reviewText                reviewText    999754      0   object
helpful_vote            helpful_vote    999754      0    int64
total_vote                total_vote    999754      0    int64
helpfulness_score  helpfulness_score    999754      0  float64

***The video_games_reviews after cleaned:***
 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量     数据类型
reviewerID                reviewerID       752      0   object
asin                            asin       752      0   object
overall                      overall       752      0  float64
reviewTime     

**Data filter**

In [30]:
video_games_reviews_filter_list=[]
for video_games_reviews in video_games_reviews_list:
    video_games_reviews_filter=filter_by_timeline(video_games_reviews)
    print('filter with timeline finished!')
    vote_analysis(video_games_reviews_filter)
    video_games_reviews_filter=filter_by_vote(video_games_reviews_filter)
    video_games_reviews_filter_list.append(video_games_reviews_filter)
    print('filter with vote finished!')

filter with timeline finished!
total vote is 0:  384880
     the ratio:  0.5674620492799094
total vote>=2:  183446
helpful vote is not 0:  223175

filter with vote finished!
filter with timeline finished!
total vote is 0:  340
     the ratio:  0.4521276595744681
total vote>=2:  285
helpful vote is not 0:  362

filter with vote finished!


In [31]:
for video_games_reviews in video_games_reviews_filter_list:
    print('\n******************\n')
    # review_len(video_games_reviews)
    data_quality(video_games_reviews,content_columns_reviews)


******************

 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量            数据类型
reviewerID                reviewerID    183446      0          object
asin                            asin    183446      0          object
overall                      overall    183446      0         float64
reviewTime                reviewTime    183446      0  datetime64[ns]
reviewText                reviewText    183446      0          object
helpful_vote            helpful_vote    183446      0           int64
total_vote                total_vote    183446      0           int64
helpfulness_score  helpfulness_score    183446      0         float64

******************

 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量            数据类型
reviewerID                reviewerID       285      0          object
asin                            asin       285      0          object
overall                      overall       2

***Concatenate***

In [32]:
video_games_reviews_concatenated = pd.concat(video_games_reviews_filter_list, ignore_index=True)
video_games_reviews_concatenated['reviewTime'] = video_games_reviews_concatenated['reviewTime'].dt.strftime('%Y-%m-%d')

In [33]:
table_name='video_games_reviews_filter'
saved_mysql(video_games_reviews_concatenated,table_name)

Table 'video_games_reviews_filter' does not exist. Creating new table...
Table 'video_games_reviews_filter' created.


In [34]:
review_len(video_games_reviews_concatenated)

Num of rows: 183731
====      
Avg sentences: 9.174913324370955
Total sentences: 1685716
====      
Max text length: 32689
Min text length: 5
Avg text length: 1015.1052081575782
Mid text length: 496
q75 text length: 1146.0


In [35]:
data_quality(video_games_reviews_concatenated,content_columns_reviews)

 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量     数据类型
reviewerID                reviewerID    183731      0   object
asin                            asin    183731      0   object
overall                      overall    183731      0  float64
reviewTime                reviewTime    183731      0   object
reviewText                reviewText    183731      0   object
helpful_vote            helpful_vote    183731      0    int64
total_vote                total_vote    183731      0    int64
helpfulness_score  helpfulness_score    183731      0  float64


# ***Books***

In [14]:
file_name='books_reviews_original'
books_reviews_list=getDF_reviews('data/reviews_Books.json.gz',review_columns,file_name)


557    Is this book out of print? Why is it only in p...
Name: reviewText, dtype: object
652    The second of the Divergent trilogy, Roth's th...
Name: reviewText, dtype: object
846    **SPOILERS! RUN AWAY!**First, let's address th...
Name: reviewText, dtype: object
950    this book surpassed my expectations.  I read D...
Name: reviewText, dtype: object
79    The negativity comes from his capitalism ideas...
Name: reviewText, dtype: object
568    Love Monkey is a first novel that works on two...
573    What's surprising about Love Monkey is not tha...
591    This book is just a dead-on, blisteringly funn...
Name: reviewText, dtype: object
187    A Crack in the Line, by Michael Lawrence, is a...
Name: reviewText, dtype: object
161    Got this book as a gift from a friend, who kn...
Name: reviewText, dtype: object
475    I read with some astonishment the 12 reviews t...
Name: reviewText, dtype: object
902    I just loved this book when I first read it se...
Name: reviewText, dtype: obje

In [15]:
print(len(books_reviews_list))

24


In [ ]:
for books_reviews in books_reviews_list:
    print('\n***The books_reviews after cleaned:***')
    # books_reviews.drop_duplicates(keep='first', inplace=True)
    # books_reviews=review_data_clean(books_reviews)
    data_quality(books_reviews,content_columns_reviews)

In [18]:
books_reviews=read_data_from_mysql('books_reviews_filter_vote_5')

Connected to MySQL


/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_70507/3048093999.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


load books_reviews_filter_vote_5 data from MySQL!


In [19]:
books_reviews.head()

,reviewerID,asin,overall,reviewTime,reviewText,helpful_vote,total_vote,helpfulness_score
0,A23PISU0ZLW71C,0000029831,5.0,2014-02-24,I thought this book was great before buying it...,5,5,1.000000
1,AF7CSSGV93RXN,000100039X,5.0,2012-03-12,I first read The Prophet by Kahlil Gibran over...,5,6,0.833333
2,A37473RZR4WOHK,000100039X,5.0,2013-03-14,While it is certainly true that Gibran was no ...,5,6,0.833333
3,A3IS4WGMFR4X65,000100039X,5.0,2009-10-27,The Prophet is Kahlil Gibran's best known work...,8,10,0.800000
4,A39FW2WG64TQGO,000100039X,2.0,2012-06-03,I'm afraid that if I write too much in this re...,2,14,0.142857


In [20]:
# vote_analysis(books_reviews)
# books_reviews=filter_by_vote(books_reviews)
len(books_reviews)

1625617

In [20]:
review_len(books_reviews)

Num of rows: 1625617
====      
Avg sentences: 9.028183760381443
Total sentences: 14676369
====      
Max text length: 37120
Min text length: 1
Avg text length: 1097.7901602899085
Mid text length: 636
q75 text length: 1355.0


In [21]:
def lan_check(text):
    try:
        if detect(text) == 'en':
            return True
        else:
            print(f'Not English!!!!!!!!!!!!!')
            return False
    except:
        print(len(text))
        return True

def del_english(df):
    # 假设df是你的DataFrame，reviewText是包含文本的列名
    # 创建一个新的列，存储每个文本的检测语言
    df['language'] = df['reviewText'].apply(lan_check)
    # 过滤掉非英语语言的数据行
    df = df[df['language']]
    # 删除辅助列
    df = df.drop(columns=['language'])
    return df

books_reviews_list=[]
for i in range(0, (len(books_reviews)//30000)+1):
    temp=books_reviews.iloc[i*30000:(i+1)*30000]
    temp = del_english(temp)
    books_reviews_list.append(temp)
    print(f'Finished:  {(i+1)*3}0,000\n     with {len(temp)} reviews')

Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!


/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_70507/198281822.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['language'] = df['reviewText'].apply(lan_check)


Finished:  30,000
     with 29983 reviews
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!


/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_70507/198281822.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['language'] = df['reviewText'].apply(lan_check)


Finished:  60,000
     with 29990 reviews
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
238


...

/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_70507/198281822.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['language'] = df['reviewText'].apply(lan_check)


Finished:  1590,000
     with 29974 reviews
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!


/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_70507/198281822.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['language'] = df['reviewText'].apply(lan_check)


Finished:  1620,000
     with 29978 reviews
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
Not English!!!!!!!!!!!!!
1
Finished:  1650,000
     with 5614 reviews


/var/folders/s9/6ly2nsx50hg7kjfnghxs0bm80000gn/T/ipykernel_70507/198281822.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['language'] = df['reviewText'].apply(lan_check)


**Data filter**

In [ ]:
total=0
books_reviews_filter_list=[]
for books_reviews in books_reviews_list:
    books_reviews_filter=filter_by_timeline(books_reviews)
    print('filter with timeline finished!')
    vote_analysis(books_reviews_filter)
    books_reviews_filter=filter_by_vote(books_reviews_filter)
    print('filter with vote finished!')
    # caculate total number of reviews
    print('\n******************\n')
    num_rows, num_cols = books_reviews_filter.shape
    total+=num_rows
    print('total: ',total)
    print('\n******************\n')
    books_reviews_filter_list.append(books_reviews_filter)

In [29]:
for books_reviews in books_reviews_filter_list:
    data_quality(books_reviews,content_columns_reviews)

 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量     数据类型
reviewerID                reviewerID   1625617      0   object
asin                            asin   1625617      0   object
overall                      overall   1625617      0  float64
reviewTime                reviewTime   1625617      0   object
reviewText                reviewText   1625617      0   object
helpful_vote            helpful_vote   1625617      0    int64
total_vote                total_vote   1625617      0    int64
helpfulness_score  helpfulness_score   1625617      0  float64


***Concatenate***

In [22]:
# books_reviews_concatenated = pd.concat(books_reviews_filter_list, ignore_index=True)

books_reviews_concatenated = pd.concat(books_reviews_list, ignore_index=True)
# make reviewTime is datetime type
books_reviews_concatenated['reviewTime'] = pd.to_datetime(books_reviews_concatenated['reviewTime'], format='%Y-%m-%d').dt.strftime('%Y-%m-%d')

In [23]:
books_reviews_concatenated.drop_duplicates(keep='first', inplace=True)

In [24]:
data_quality(books_reviews_concatenated,content_columns_reviews)

 data of specified columns is NaN: 0
                                  列名  num_rows  缺失值数量     数据类型
reviewerID                reviewerID   1623922      0   object
asin                            asin   1623922      0   object
overall                      overall   1623922      0  float64
reviewTime                reviewTime   1623922      0   object
reviewText                reviewText   1623922      0   object
helpful_vote            helpful_vote   1623922      0    int64
total_vote                total_vote   1623922      0    int64
helpfulness_score  helpfulness_score   1623922      0  float64


In [25]:
# table_name='books_reviews_filter'
table_name='books_reviews_filter_final'
saved_mysql(books_reviews_concatenated,table_name)

Table 'books_reviews_filter_final' does not exist. Creating new table...
Table 'books_reviews_filter_final' created.


In [26]:
review_len(books_reviews_concatenated)

Num of rows: 1623922
====      
Avg sentences: 9.033063164363806
Total sentences: 14668990
====      
Max text length: 33036
Min text length: 1
Avg text length: 1098.257458794203
Mid text length: 636.0
q75 text length: 1356.0


# **Load Product Data and Do Data Clean**

# ***Cell Phone and Accessories***

In [ ]:
cell_phones_meta= getDF('data/meta_Cell_Phones_and_Accessories.json.gz',meta_columns)
cell_phones_meta.head()

In [ ]:
table_name='cell_phones_meta_original'
saved_mysql(cell_phones_meta,table_name)

In [ ]:
cell_phones_meta_report = data_quality(cell_phones_meta, content_columns_meta)
cell_phones_meta_report

***delete NaN***

In [ ]:
cell_phones_meta=meta_delete_all_NaN(df=cell_phones_meta,table_name=table_name,content_columns=content_columns_meta)

In [ ]:
cell_phones_meta_report=data_quality(cell_phones_meta,content_columns_meta)
cell_phones_meta_report

In [ ]:
table_name='cell_phones_meta_cleaned'
saved_mysql(cell_phones_meta,table_name)

# ***Electronics***

In [ ]:
electronics_meta= getDF('data/meta_Electronics.json.gz',meta_columns)
electronics_meta.head()

In [ ]:
table_name='electronics_meta_original'
saved_mysql(electronics_meta,table_name)

In [ ]:
electronics_meta_report = data_quality(electronics_meta, content_columns_meta)
electronics_meta_report

***delete NaN***

In [ ]:
electronics_meta=meta_delete_all_NaN(df=electronics_meta,table_name=table_name,content_columns=content_columns_meta)

In [ ]:
table_name='electronics_meta_cleaned'
saved_mysql(electronics_meta,table_name)

# ***Books***

In [ ]:
books_meta= getDF('data/meta_Books.json.gz',meta_columns)
books_meta.head()

In [ ]:
table_name='books_meta_original'
saved_mysql(books_meta,table_name)

In [ ]:
# books_meta=read_data_from_mysql('books_meta_original')

In [ ]:
data_quality(books_meta,content_columns_meta)

In [ ]:
books_meta=meta_delete_all_NaN(df=books_meta,table_name=table_name,content_columns=content_columns_meta)

In [ ]:
data_quality(books_meta,content_columns_meta)

In [ ]:
table_name='books_meta_cleaned'
saved_mysql(books_meta,table_name)

***Video Games***

In [ ]:
video_games_meta= getDF('data/meta_Video_Games.json.gz',meta_columns)
video_games_meta.head()

In [ ]:
table_name='video_games_meta_original'
saved_mysql(video_games_meta,table_name)

In [ ]:
data_quality(video_games_meta,content_columns_meta)

In [ ]:
video_games_meta=meta_delete_all_NaN(df=video_games_meta,table_name=table_name,content_columns=content_columns_meta)

In [ ]:
data_quality(video_games_meta,content_columns_meta)

In [ ]:
table_name='video_games_meta_cleaned'
saved_mysql(video_games_meta,table_name)

# *product clean*

In [ ]:
meta_cell_phones=read_data_from_mysql('meta_cell_phones')
meta_electronics=read_data_from_mysql('meta_electronics')
meta_books=read_data_from_mysql('meta_books')
meta_video_games=read_data_from_mysql('meta_video_games')

In [ ]:
data_quality(meta_cell_phones,content_columns_meta)
data_quality(meta_electronics,content_columns_meta)
data_quality(meta_books,content_columns_meta)
data_quality(meta_video_games,content_columns_meta)

**SP**

In [ ]:
cell_phones_reviews=read_data_from_mysql('cell_phones_reviews_filter')
electronics_reviews=read_data_from_mysql('electronics_reviews_filter')

In [ ]:
data_quality(cell_phones_reviews,content_columns_reviews)
data_quality(electronics_reviews,content_columns_reviews)

In [ ]:
asin_cell_phone = cell_phones_reviews['asin'].drop_duplicates().reset_index(drop=True)
asin_electronics = electronics_reviews['asin'].drop_duplicates().reset_index(drop=True)
asin_electronics.head()

In [ ]:
# Convert the Series to a DataFrame
asin_cell_phone = pd.DataFrame({'asin': asin_cell_phone})
asin_electronics=pd.DataFrame({'asin': asin_electronics})

In [ ]:
asin_cell_phone.columns

In [ ]:
len(asin_electronics)

In [ ]:
# Filter 'meta_cell_phone' based on whether 'asin' values are present in 'product'
meta_cell_phone = meta_cell_phones[meta_cell_phones['asin'].isin(asin_cell_phone['asin'])]
file_name='meta_cell_phone_filter'
saved_mysql(meta_cell_phone,file_name)
data_quality(meta_cell_phone,content_columns_meta)

In [ ]:
meta_electronics = meta_electronics[meta_electronics['asin'].isin(asin_electronics['asin'])]
file_name='meta_electronics_filter'
saved_mysql(meta_electronics,file_name)
data_quality(meta_electronics,content_columns_meta)

*EP*

In [ ]:
video_games_reviews=read_data_from_mysql('video_games_reviews_filter')
data_quality(video_games_reviews,content_columns_reviews)

In [ ]:
asin_video_games = video_games_reviews['asin'].drop_duplicates().reset_index(drop=True)
asin_video_games = pd.DataFrame({'asin': asin_video_games})
asin_video_games.head()

In [ ]:
len(asin_video_games)

In [ ]:
meta_video_games = meta_video_games[meta_video_games['asin'].isin(asin_video_games['asin'])]
file_name='meta_video_games_filter'
saved_mysql(meta_video_games,file_name)
data_quality(meta_video_games,content_columns_meta)

In [ ]:
books_reviews=read_data_from_mysql('books_reviews_filter')
data_quality(books_reviews,content_columns_reviews)

In [ ]:
asin_books = books_reviews['asin'].drop_duplicates().reset_index(drop=True)
asin_books = pd.DataFrame({'asin': asin_books})

In [ ]:
len(asin_books)

In [ ]:
meta_books=read_data_from_mysql('books_meta_original')

In [ ]:
data_quality(meta_books,content_columns_meta)

In [ ]:
table_name='books_meta_original'
meta_books=meta_delete_all_NaN(df=meta_books,table_name=table_name,content_columns=content_columns_meta)

In [ ]:
data_quality(meta_books,content_columns_meta)

In [ ]:
meta_books = meta_books[meta_books['asin'].isin(asin_books['asin'])]

In [ ]:
duplicates = asin_books.duplicated()
duplicates[duplicates == True]

In [ ]:
# meta_books = meta_books[meta_books['asin'].isin(asin_books['asin'])]
file_name='meta_books_filter'
saved_mysql(meta_books,file_name)
data_quality(meta_books,content_columns_meta)